In [0]:
processName = dbutils.widgets.get('prm_processName')

nextSourceFileDate = f"""SELECT MAX(PROCESSED_FILE_TABLE_DATE)+1 as NEXT_SOURCE_FILE_DATE FROM processrunlogs.deltalakehouse_process_runs
where PROCESS_NAME = '{processName}' and PROCESS_STATUS = 'Completed' """

nextSourceFileDateDF = spark.sql(nextSourceFileDate)


In [0]:
#Source URL for the data
# sourceFileURL = 'https://retailpricing.blob.core.windows.net/labs/lab1/PW_MW_DR_01012023.csv'
# bronzelayerCSVFilePath = 'abfss://working-labs@pricepredictionadls.dfs.core.windows.net/bronze/daily-pricing/csv'

In [0]:
# dailyPricingSourceBaseURL = 'https://retailpricing.blob.core.windows.net/'
# dailyPricingSourceFolder = 'daily-pricing/'
# daiilyPricingSourceFileDate = datetime.strptime(str(nextSourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']),'%Y-%m-%d').strftime('%m%d%Y')
# daiilyPricingSourceFileName = f"PW_MW_DR_{daiilyPricingSourceFileDate}.csv"


# daiilyPricingSinkLayerName = 'bronze'
# daiilyPricingSinkStorageAccountName = 'pricepredictionadls'
# daiilyPricingSinkFolderName =  'daily-pricing'

In [0]:
from datetime import datetime


In [0]:
#modularize the URL of Source and sink path
dailyPricingSourceBaseURL = 'https://retailpricing.blob.core.windows.net/'
dailyPricingSourceFolder = 'daily-pricing/'
#dailyPricingSourceFileDate=dbutils.widgets.get('prm_dailyPricingSourceFileDate')
daiilyPricingSourceFileDate = datetime.strptime(str(nextSourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']),'%Y-%m-%d').strftime('%m%d%Y')
daiilyPricingSourceFileName = f'PW_MW_DR_{dailyPricingSourceFileDate}.csv'



daiilyPricingSinkLayerName = 'bronze'
daiilyPricingSinkStorageAccountName = 'pricepredictionadls'
daiilyPricingSinkFolderName =  'daily-pricing'

In [0]:
import pandas as pds

In [0]:
#Variable for source path
dailyPricingSourceURL = dailyPricingSourceBaseURL + dailyPricingSourceFolder + daiilyPricingSourceFileName

#read the csv file from source  in the  pandas dataframe
dailyPricingPandasDF=pds.read_csv(dailyPricingSourceURL)


In [0]:
#Convert the pandas dataframe into spark dataframe
dailyPricingSparkDF=spark.createDataFrame(dailyPricingPandasDF)

In [0]:

from pyspark.sql.functions import current_timestamp
#Writing the Spark dataframe into Bronze layer
dailyPricingSinkFolderPath=f"abfss://{daiilyPricingSinkLayerName}@{daiilyPricingSinkStorageAccountName}.dfs.core.windows.net/"

(
    dailyPricingSparkDF.withColumn("source_file_load_date", current_timestamp()).write.mode("append").option("header", "true").csv(dailyPricingSinkFolderPath)
)

In [0]:
dbutils.fs.rm(dailyPricingSinkFolderPath)

In [0]:
%sql
use catalog `price-prediction-analytics`;

CREATE schema if not exists processrunlogs;

CREATE TABLE IF NOT EXISTS processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(
  PROCESS_NAME STRING,
  PROCESSED_FILE_TABLE_DATE DATE,
  PROCESS_STATUS string
)


In [0]:
processName= 'dailyPricingSourceIngest'
processFileDate =  dbutils.widgets.get('prm_dailyPricingSourceFileDate')
processStatus= 'Completed'

processInsertSql=f""" INSERT INTO processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS VALUES('{processName}','{processFileDate}','{processStatus}')"""
print(processInsertSql)
spark.sql(processInsertSql)

In [0]:
%sql
SELECT * FROM processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS

In [0]:
%sql
SELECT MAX(PROCESSED_FILE_TABLE_DATE)+1 as NEXT_SOURCE_FILE_DATE FROM processrunlogs.deltalakehouse_process_runs
where PROCESS_NAME = 'dailyPricingSourceIngest' and PROCESS_STATUS = 'Completed'